In [ ]:
import os
from api import upload_data, upload_dummy_data, start_trainer, create_prediction, get_results, delete_data, CreatePredictionResponseSuccess
TENANT_ID = os.getenv("TENANT_ID")

### Basic flow
First, we create a simple dummy dataset. This will have dataset_id "dummy-dataset-1". `upload_dummy_data()` both generate the data and uploads it to the s3 bucket.

In [ ]:
dummy_dataset_ids = upload_dummy_data(TENANT_ID, nbr_datasets=2)
print(f"Datasets {dummy_dataset_ids} were uploaded")

Next step is to train a model on the data.

In [ ]:
start_trainer(TENANT_ID, dummy_dataset_ids)

When the training is complete, the prediction can be made.

In [ ]:
res: CreatePredictionResponseSuccess = create_prediction(TENANT_ID, dummy_dataset_ids)
results = get_results(TENANT_ID, job_id=res.jobId).results

In [ ]:
for result in results:
    print(f"Dataset ID: {result.datasetId}")
    print("Replenishment Suggestion:", result.replenishmentSuggestion.quantity)
    print("Suggested safety stock:", result.safetyStockSuggestion.quantity)
    print("Suggested reorder point:", result.reorderPointSuggestion.quantity)
    print() 

Finally, we delete the data again

In [ ]:
for dataset_id in dummy_dataset_ids:
    delete_data(TENANT_ID, dataset_id=dataset_id)

### Upload you own data
In order upload data instead of the randomly created data, the function `upload_data()` can be used. This function takes the data path as input.
The data file should be a .json file, and an example of a file is placed in "data/data_example.json". 

The `datasets` consists of a `datasetId` and a list of transactions. The `transactions` array has has three required fields: 
- <b>quantity</b>: The number of items in that transaction.
- <b>departureDate</b>: The date the item was shipped out of the warehouse.
- <b>transactionId</b>: An unique transactionId.

Besides that the `transactions` have optional fields
- <b>unitPrice</b>: The price the shop charges for the product.
- <b>unitCost</b>: The cost the shop paid their supplier for the product.

In [ ]:
example_dataset_ids = upload_data(TENANT_ID, path="data/data_example.json")
print(f"Datasets {example_dataset_ids} were uploaded")

Now we can run the training and prediction exactly as before.

In [ ]:
start_trainer(TENANT_ID, example_dataset_ids)
res: CreatePredictionResponseSuccess = create_prediction(TENANT_ID, example_dataset_ids)
results = get_results(TENANT_ID, job_id=res.jobId).results
for result in results:
    print(f"Dataset ID: {result.datasetId}")
    print("Replenishment Suggestion:", result.replenishmentSuggestion.quantity)
    print("Suggested safety stock:", result.safetyStockSuggestion.quantity)
    print("Suggested reorder point:", result.reorderPointSuggestion.quantity)
    print() 

In [ ]:
for dataset_id in example_dataset_ids:
    delete_data(TENANT_ID, dataset_id=dataset_id)